# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [3]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [4]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財金新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [8]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            print('到達頁面底端')
            break
        print('網頁更新中...')
        last_height = new_height
        
    #提示：可參考以下的Stack Overflow: 
    #https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [9]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [10]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 孫正義宣布與比亞迪合作　5月起...免費提供「3億片口罩」

▲中國汽車大廠比亞迪，迅速轉產口罩。（圖／翻攝　微博　IT 之家）
記者任以芳／綜合報導
新冠肺炎（COVlD-19）疫情衝擊日本，尤其東京最為嚴重。根據路透社報導，軟體銀行集團創辦人孫正義(Masayoshi Son)表示，將與全球最大口罩中國廠商比亞迪（BYD）合作，預計月產3億片口罩，未來將捐出第一線日本醫療人員。
▲日本軟銀創辦人孫正義宣布與中國比亞迪合作生產口罩。（資料圖／記者呂佳賢攝） 
孫正義11日在個人推特（Twitter）留言，軟銀將與日本政府的口罩小組合作，捐贈兩種不同的口罩，首先為醫務人員準備的口罩，該口罩小組的成立，是為了解決因疫情影響的口罩荒。
合作廠商中國比亞迪也證實這項合作，正在建立一條新的生產線來生產口罩。比亞迪發言人表示，公司產能已經達到每天1500萬個口罩，「未來，針對軟銀的供應將包括1億個N95口罩和2億個普通外科口罩，其中N95口罩可以過濾非常小的顆粒。」
▲疫情期間，中國比亞迪斜槓發展，創下驚人的日產速度。（圖／翻攝　微博 c科技）
據了解，比亞迪是在香港上市的中國民營企業，總部設於廣東深圳，董事長兼總裁是王傳福，主要從事電池、汽車及代工廠，從2月份開始，比亞迪迅速轉產口罩，創造驚人速度。
根據大陸媒體指出，疫情爆發後，比亞迪上千名工程師齊心研發，3天畫出400多張設備圖紙，7天時間完成口罩機生產設備的研發製造。比亞迪在30天內建成全球最大的口罩產能，截至4月初的口罩日產能已經超過1千萬片。相較市面上，造一台口罩機，快則要15天，慢則要30天。
----------------------------------------------------------------------

[1] 加油要快！油價明天調漲0.6元「連跌六周反彈」

▲全球各主要產油國召開會議談減產，帶動國際油價走揚，台灣中油今（12）日公佈調漲下周國內汽柴油零售價格。（圖／記者林敬旻攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，加上沙俄等產油國重新協商減產協議，牽動國際油價近日走揚，台灣中油今（12）日公佈下周國內汽